이 노트는 캐글의 "Toxic Comment Classification Challenge"를 예제로 케라스 활용법을 익히는 데 주안점이 있으며, [1. 간단한 모델 만들기](https://github.com/Kim-Hohyun/Toxic-Comment-Classification-Challenge/blob/master/1.%20%EA%B0%84%EB%8B%A8%ED%95%9C%20%EB%AA%A8%EB%8D%B8%20%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb)에 이어지는 내용을 담고 있습니다. 

# 첫 번째 모델

In [1]:
# 사용할 모듈 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras import Input, models, layers, callbacks
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import roc_auc_score
from keras.utils import plot_model

Using TensorFlow backend.


In [2]:
data = pd.read_csv('train.csv')
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


각 클래스 별로 비율을 확인합니다. 'threat' 클래스의 비율이 특히 낮습니다. 

In [3]:
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
data[classes].sum() / len(data)

toxic            0.095844
severe_toxic     0.009996
obscene          0.052948
threat           0.002996
insult           0.049364
identity_hate    0.008805
dtype: float64

훈련셋과 검증셋을 나눌 때 'threat'의 비율을 맞춰주겠습니다. 

In [0]:
train_data, val_data = train_test_split(data, 
                                        test_size=0.2,
                                        stratify=data['threat'], 
                                        random_state=42)

전처리 방식은 이전과 거의 같습니다.

In [0]:
tokenizer = Tokenizer(num_words = 5000,
                      lower=True,
                      char_level=True,
                      oov_token=True)   # 빈도수가 낮은 단어(상위 4998에 속하지 않는 단어)는 모두 하나의 단어처럼 처리해줍니다.

tokenizer.fit_on_texts(train_data['comment_text'])

# 상위 4998개의 단어에 대해 2~4999의 자연수를 매핑하고, 1은 oov_token에 배정됩니다. 0은 패딩용입니다.
sequences = tokenizer.texts_to_sequences(train_data['comment_text']) 
val_sequences = tokenizer.texts_to_sequences(val_data['comment_text'])
X_train = pad_sequences(sequences, maxlen=250)
X_val = pad_sequences(val_sequences, maxlen=250)

y_train = np.asarray(train_data[classes])
y_val = np.asarray(val_data[classes])

"Toxic Comment Classification Challenge"는 각 클래스 별 roc-auc의 평균을 평가 지표로 사용합니다. 모델을 적합시킬 때 변화하는 roc-auc를 관찰하기 위해서 나만의 콜백을 정의합니다.

In [0]:
class Histories(callbacks.Callback):

  # 에포크가 끝날 때 아래 콜백이 호출됩니다.
  def on_epoch_end(self, epoch, logs={}):
    y_pred = self.model.predict(self.validation_data[0]) # self.validation_data를 이용해서 검증셋의 샘플과 레이블을 리스트로 부를 수 있습니다.
    _auc_val = roc_auc_score(self.validation_data[1], y_pred)
    print(' - auc_val: %s' % str(round(_auc_val, 4)))
    return

# 사용할 콜백을 하나의 리스트로 묶습니다. EarlyStopping 콜백은 검증 손실값이 떨어지지 않을 때 자동으로 훈련을 멈춥니다.
callbacks_list = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=1,
    ),
    Histories()
]


여기서는 Con1D를 사용하겠습니다.

In [0]:
input_tensor = Input(shape=(250,))
x = layers.Embedding(5000, 128)(input_tensor)

x = layers.Conv1D(256, 5, padding='same', activation='relu')(x)
x = layers.Conv1D(256, 5, padding='same', activation='relu')(x)
x = layers.MaxPooling1D(2)(x)

x = layers.Conv1D(512, 7, padding='same', activation='relu')(x)
x = layers.Conv1D(512, 7, padding='same', activation='relu')(x)
x = layers.MaxPooling1D(5)(x)

x = layers.Flatten()(x)
x = layers.Dense(1024, activation='sigmoid')(x) # relu로 하면 성능이 안 나옵니다. 이유는 모르겠습니다... 
output_tensor = layers.Dense(6, activation='sigmoid')(x) # 6개의 클래스 별 확률을 구해야 하기 때문에 마지막 층의 아웃풋은 6으로 지정합니다.

model = models.Model(input_tensor, output_tensor)

In [0]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy')

In [25]:
history = model.fit(X_train, y_train, epochs=30, batch_size=256, 
                    validation_data=(X_val, y_val),
                    callbacks=callbacks_list)

Train on 127656 samples, validate on 31915 samples
Epoch 1/30
127656/127656 [==============================] - 174s 1ms/step - loss: 0.1016 - val_loss: 0.0700
 - auc_val: 0.9417
Epoch 2/30
127656/127656 [==============================] - 173s 1ms/step - loss: 0.0632 - val_loss: 0.0538
 - auc_val: 0.9694
Epoch 3/30
127656/127656 [==============================] - 173s 1ms/step - loss: 0.0545 - val_loss: 0.0552
 - auc_val: 0.9722


# 두 번째 모델

두 번째 모델은 모델의 층은 첫 번째 모델과 거의 같게 만들고 전처리 방식을 다르게 해보겠습니다. 여기서는 단어 대신 개별 문자(a, b, c, ...)를 토큰화합니다.


In [0]:
char_tokenizer = Tokenizer(num_words = 60,
                      lower=True,
                      char_level=True,
                      oov_token=True)  

char_tokenizer.fit_on_texts(train_data['comment_text'])

char_sequences = char_tokenizer.texts_to_sequences(train_data['comment_text']) 
char_val_sequences = char_tokenizer.texts_to_sequences(val_data['comment_text'])
char_X_train = pad_sequences(char_sequences, maxlen=800)
char_X_val = pad_sequences(char_val_sequences, maxlen=800)


X_train, X_val = text_to_array()

In [45]:
input_tensor = Input(shape=(800,))
x = layers.Embedding(60, 128)(input_tensor)

x = layers.Conv1D(256, 5, padding='same', activation='relu')(x)
x = layers.Conv1D(256, 5, padding='same', activation='relu')(x)
x = layers.MaxPooling1D(5)(x)

x = layers.Conv1D(512, 7, padding='same', activation='relu')(x)
x = layers.Conv1D(512, 7, padding='same', activation='relu')(x)
x = layers.MaxPooling1D(5)(x)

x = layers.Flatten()(x)
x = layers.Dense(1024, activation='sigmoid')(x) 
output_tensor = layers.Dense(6, activation='sigmoid')(x) 

char_model = models.Model(input_tensor, output_tensor)
char_model.compile(optimizer='rmsprop', loss='binary_crossentropy') 

char_history = char_model.fit(char_X_train, y_train, epochs=30, batch_size=256, 
                    validation_data=(char_X_val, y_val),
                    callbacks=callbacks_list)

Train on 127656 samples, validate on 31915 samples
Epoch 1/30
127656/127656 [==============================] - 328s 3ms/step - loss: 0.1071 - val_loss: 0.0755
 - auc_val: 0.9444
Epoch 2/30
127656/127656 [==============================] - 326s 3ms/step - loss: 0.0662 - val_loss: 0.0622
 - auc_val: 0.9647
Epoch 3/30
127656/127656 [==============================] - 326s 3ms/step - loss: 0.0562 - val_loss: 0.0509
 - auc_val: 0.9739
Epoch 4/30
127656/127656 [==============================] - 326s 3ms/step - loss: 0.0515 - val_loss: 0.0501
 - auc_val: 0.9765
Epoch 5/30
127656/127656 [==============================] - 326s 3ms/step - loss: 0.0482 - val_loss: 0.0556
 - auc_val: 0.9722


# 앙상블

In [47]:
# 검증셋에 대한 두 모델의 roc_auc 수치가 비슷하게 나왔다.
model1_pred = model.predict(X_val)
model2_pred = char_model.predict(char_X_val)

roc_auc_score(y_val, model1_pred), roc_auc_score(y_val, model2_pred)

(0.9722092430202095, 0.9721930718295222)

In [64]:
# 두 모델의 예측 확률을 평균내어 roc_auc 수치를 구해보자.
ensemble_model = (model1_pred + model2_pred) / 2
roc_auc_score(y_val, ensemble_model)

0.9776605169387826

성능이 향상되었다!